In [1]:
import diffusers
import torch
import torch.nn as nn
import sys
sys.path.append("../")

from dataclasses import dataclass
import dataclasses
from torchvision import transforms
from DDPMPipelineDropout import DDPMPipeline

import utils

In [2]:
model, config = utils.load_model("ddpm-ffhq-128/unet/DDPM-FFHQ-10-PERCENT-DROPOUT-DOWN-MID-UP-10-SAMPLES-epoch-0.pth")
config.bayesian_avg_samples = 1
scheduler = diffusers.DDPMScheduler(num_train_timesteps=config.num_train_timestamps)
pipeline = DDPMPipeline(unet=model, scheduler=scheduler).to("cuda")

In [3]:
fid_score, inception_score = utils.calculate_metrics(config, pipeline, batch_size=1, num_images=4)

c:\Users\bhata\anaconda3\envs\pytorch\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `FrechetInceptionDistance` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
c:\Users\bhata\anaconda3\envs\pytorch\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
4it [11:18, 169.66s/it]

Image Limit reached breaking


In [4]:
fid_score

tensor(408.0741)

In [5]:
inception_score

(tensor(1.), tensor(5.9605e-08))